In [1]:
# Web Scraping
!pip install BeautifulSoup4
!pip install requests
!pip install lxml

In [2]:
from bs4 import BeautifulSoup

In [3]:

import requests
import pandas as pd
import numpy as np
from lxml import html
import lxml.etree

print('libraries imported!')


# get html from wiki, create Soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").content
soup = BeautifulSoup(source, "html.parser")
print('soup object created')


# iterate wikitable to get data from HTML, store to list
data = []
table = soup.find(class_='wikitable')
rows = table.find_all('tr')

for row in rows:
    data.append([td.text.strip() for td in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.head()

libraries imported!
soup object created


,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [4]:
# clean data
# clean Borough = Not Assigned or Neightbourhood is blank
df['Neighbourhood'].replace('', np.nan, inplace=True)
df.head()


# Remove rows with NaN
df.dropna(subset=['Neighbourhood'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(20)


df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [5]:
df.shape

(103, 3)

### End of part 1

In [6]:
# reset index to PostalCode for join table later
df = df.set_index('PostalCode')
df.head()

# Add Geospatial Data

#Get data lat/long data from csv
coor_df = pd.read_csv('http://cocl.us/Geospatial_data')

#rename columns and set the index to be Postcode
coor_df.columns = ["PostalCode", "Latitude", "Longitude"]
if(coor_df.index.name != 'PostalCode'):
    coor_df = coor_df.set_index('PostalCode')
    
coor_df

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


In [7]:
df = df.join(coor_df).reset_index()
df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### End of part 2

In [21]:
# download all dependencies
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [22]:
# get Borough that contains Toronto Data
df = df[df['Borough'].str.contains('Toronto')]


In [23]:
df.head(5)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [24]:
# create map of Toronto areas
address = "Toronto, ON"



In [25]:
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude



In [26]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
map_toronto

# add markers to map for the Neighbourhoods in Toronto
for lat, lng, borough, neighbourhood in zip(
        df['Latitude'], 
        df['Longitude'], 
        df['Borough'], 
        df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### The End